In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import math
import logging
sys.path.append('../')

#import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb
#from process.footprint_report import update_ln, update_poly, update_pt
from shapely import Polygon, MultiPolygon
from shapely import affinity
#import dask_geopandas

from its_logging.logger_config import logger
logger = logging.getLogger('process.footprint')

In [2]:
upload_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload"

In [48]:
os.listdir(upload_path)

['ITS_V2.0_Activity.gdb',
 'ITS_V2_0_Footprint_Points.gdb',
 'ITS_V2_0_Footprint_Polygons.gdb',
 'ITS_V2_0_Lines.gdb',
 'ITS_V2_0_Points.gdb',
 'ITS_V2_0_Polygons.gdb']

In [49]:
for f in os.listdir(upload_path):
    gdb_layer = gpd.list_layers(os.path.join(upload_path,f)).loc[0, 'name']
    gdf = gpd.read_file(os.path.join(upload_path,f), driver='OpenFileGDB', layer=gdb_layer)
    gdf.geometry = gdf.make_valid()
    if len(gdf.geom_type.unique()) > 1:
        print("Found multiple geom types")
        break
    save_gdf_to_gdb(gdf,os.path.join(upload_path,f),gdb_layer)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-27 12:04:21,192 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:04:21,192 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:04:21,203 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Lines.gdb


2025-08-27 12:04:41,099 INFO  [pyogrio._io           ]  Created 131,658 records
2025-08-27 12:04:45,052 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Lines.gdb ITS_V2_0_Lines
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-27 12:04:48,145 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:04:48,145 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:04:48,147 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Points.gdb


2025-08-27 12:04:49,100 INFO  [pyogrio._io           ]  Created 20,443 records
2025-08-27 12:04:49,240 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Points.gdb ITS_V2_0_Points
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


Found multiple geom types


In [59]:
gdf.geom_type.unique()

array(['MultiPolygon'], dtype=object)

In [56]:
def get_poly(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    elif isinstance(geometry, MultiPolygon):
        return geometry
    else:
        out = []
        for geom in geometry.geoms:
            if isinstance(geom, Polygon):
                out.append(geom)
            elif isinstance(geom, MultiPolygon):
                for g in geom.geoms:
                    out.append(g)
            else:
                continue
        
        return MultiPolygon(out)
        

In [57]:
gdf.geometry = gdf.geometry.apply(get_poly)

In [58]:
np.all(gdf.is_valid)

np.True_

In [60]:
save_gdf_to_gdb(gdf,os.path.join(upload_path,f),gdb_layer)

2025-08-27 12:12:00,993 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:12:00,993 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:12:01,030 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Polygons.gdb


2025-08-27 12:12:28,509 INFO  [pyogrio._io           ]  Created 454,597 records
2025-08-27 12:12:35,637 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Polygons.gdb ITS_V2_0_Polygons


In [52]:
for f in os.listdir(upload_path):
    gdb_layer = gpd.list_layers(os.path.join(upload_path,f)).loc[0, 'name']
    gdf = gpd.read_file(os.path.join(upload_path,f), driver='OpenFileGDB', layer=gdb_layer)
    print(np.all(gdf.is_valid))

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


...

True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


False


In [53]:
gdb_layer

'ITS_V2_0_Polygons'

In [54]:
gdf.geometry = gdf.make_valid()

In [55]:
gdf.geom_type.unique()

array(['MultiPolygon', 'GeometryCollection'], dtype=object)